# Load 10x RNA-seq gene expression data 

In [18]:
import os
os.chdir("..")
import pandas as pd
import numpy as np
import anndata
import time
import json
import requests
from Utils.Settings import root_data, version

In [19]:

download_base = f'{root_data}/abc_download_root'

manifest_path = 'releases/%s/manifest.json' % version

url = 'https://allen-brain-cell-atlas.s3-us-west-2.amazonaws.com/' + manifest_path
manifest = json.loads(requests.get(url).text)

metadata = manifest['file_listing']['WMB-10X']['metadata']

In [3]:
view_directory = os.path.join( download_base, 
                               manifest['directory_listing']['WMB-10X']['directories']['metadata']['relative_path'], 
                              'views')
cache_views = False
if cache_views :
    os.makedirs( view_directory, exist_ok=True )

In [4]:
rpath = metadata['cell_metadata']['files']['csv']['relative_path']
file = os.path.join( download_base, rpath)
cell = pd.read_csv(file)
cell.set_index('cell_label',inplace=True)

### Gene expression matrices

The large 4 million cell dataset has been divided into 23 packages to make data transfer and download more efficient. Each package is formatted as annadata h5ad file with minimal metadata. In this next section, we provide example code on how to open the file and connect with the rich cell level metadata discussed above.

For each subset, there are two h5ad files one storing the raw counts and the other log normalization of it.

In [5]:
matrices = cell.groupby(['dataset_label','feature_matrix_label'])[['library_label']].count()
matrices.columns  = ['cell_count']
matrices

cell_count
dataset_label feature_matrix_label             
WMB-10XMulti  WMB-10XMulti                 1687
WMB-10Xv2     WMB-10Xv2-CTXsp             43985
              WMB-10Xv2-HPF              207281
              WMB-10Xv2-HY                99879
              WMB-10Xv2-Isocortex-1      248776
              WMB-10Xv2-Isocortex-2      249360
              WMB-10Xv2-Isocortex-3      249356
              WMB-10Xv2-Isocortex-4      248784
              WMB-10Xv2-MB                29781
              WMB-10Xv2-OLF              192182
              WMB-10Xv2-TH               130555
WMB-10Xv3     WMB-10Xv3-CB               181723
              WMB-10Xv3-CTXsp             78223
              WMB-10Xv3-HPF              181055
              WMB-10Xv3-HY               162296
              WMB-10Xv3-Isocortex-1      227670
              WMB-10Xv3-Isocortex-2      227537
              WMB-10Xv3-MB               337101
              WMB-10Xv3-MY               191746
              WMB-10Xv3-OLF               88560
              WMB-10Xv3-P                143157
              WMB-10Xv3-PAL              108046
              WMB-10Xv3-STR              283782
              WMB-10Xv3-TH               130454

### Example use cases

In this section, we explore two use cases. The first example looks at the expression of nine canonical neurotransmitter transporter genes and the second the expression of gene Tac2.

To support these use cases, we will create a smaller submatrix (all cells and 10 genes) and write to file for resue in part 2b. *Note this operation takes around 5 minutes*.

In [6]:
matrix_label = matrices.index[0][1]
dataset_label = matrices.index[0][0]

expression_matrices = manifest['file_listing'][dataset_label]['expression_matrices']
print(expression_matrices[matrix_label])

{'log2': {'files': {'h5ad': {'version': '20230830', 'relative_path': 'expression_matrices/WMB-10XMulti/20230830/WMB-10XMulti-log2.h5ad', 'url': 'https://allen-brain-cell-atlas.s3.us-west-2.amazonaws.com/expression_matrices/WMB-10XMulti/20230830/WMB-10XMulti-log2.h5ad', 'size': 89318511}}}, 'raw': {'files': {'h5ad': {'version': '20230830', 'relative_path': 'expression_matrices/WMB-10XMulti/20230830/WMB-10XMulti-raw.h5ad', 'url': 'https://allen-brain-cell-atlas.s3.us-west-2.amazonaws.com/expression_matrices/WMB-10XMulti/20230830/WMB-10XMulti-raw.h5ad', 'size': 132220015}}}}


In [7]:
rpath = expression_matrices[matrix_label]['log2']['files']['h5ad']['relative_path']
file = os.path.join( download_base, rpath)
print(file)

/alzheimer/Roberto/Allen_Institute/abc_download_root/expression_matrices/WMB-10XMulti/20230830/WMB-10XMulti-log2.h5ad


In [8]:
ad = anndata.read_h5ad(file,backed='r')
gene = ad.var

In [9]:
gene

,gene_symbol
gene_identifier,
ENSMUSG00000051951,Xkr4
ENSMUSG00000089699,Gm1992
ENSMUSG00000102331,Gm19938
ENSMUSG00000102343,Gm37381
ENSMUSG00000025900,Rp1
...,...
ENSMUSG00000095523,AC124606.1
ENSMUSG00000095475,AC133095.2
ENSMUSG00000094855,AC133095.1


In [11]:

gnames = ad.var.gene_symbol[(ad.var.gene_symbol.str.contains("Htr")) & (~ad.var.gene_symbol.str.contains("Htra"))].values
pred = [x in gnames for x in gene.gene_symbol]
gene_filtered = gene[pred]
gene_filtered

,gene_symbol
gene_identifier,
ENSMUSG00000026228,Htr2b
ENSMUSG00000050534,Htr5b
ENSMUSG00000070687,Htr1d
ENSMUSG00000028747,Htr6
ENSMUSG00000039106,Htr5a
ENSMUSG00000032269,Htr3a
ENSMUSG00000008590,Htr3b
ENSMUSG00000049511,Htr1b
ENSMUSG00000021721,Htr1a


In [45]:
# create empty gene expression dataframe
gdata = pd.DataFrame(index=cell.index,columns=gene_filtered.index)
count = 0
total_start = time.process_time()

for matindex in matrices.index :
    
    ds = matindex[0]
    mp = matindex[1]
    
    print(mp)
    
    expression_matrices = manifest['file_listing'][ds]['expression_matrices']
    rpath = expression_matrices[mp]['log2']['files']['h5ad']['relative_path']
    file = os.path.join( download_base, rpath)
    
    pred = (cell['feature_matrix_label'] == mp)
    cell_filtered = cell[pred]
    
    start = time.process_time()
    ad = anndata.read_h5ad(file,backed='r')
    exp = ad[cell_filtered.index, gene_filtered.index].to_df()
    gdata.loc[ exp.index, gene_filtered.index ] = exp
    print(" - time taken: ", time.process_time() - start)
    
    ad.file.close()
    del ad
    
    count += 1
    
    #if count > 2 :
    #    break
        
print("total time taken: ", time.process_time() - total_start)
    

WMB-10XMulti
 - time taken:  2.675148586999967
WMB-10Xv2-CTXsp
 - time taken:  9.192121081999858
WMB-10Xv2-HPF
 - time taken:  42.59852547599985
WMB-10Xv2-HY
 - time taken:  13.203141288000097
WMB-10Xv2-Isocortex-1
 - time taken:  66.39969433800002
WMB-10Xv2-Isocortex-2
 - time taken:  72.90410177900003
WMB-10Xv2-Isocortex-3
 - time taken:  62.07256603199994
WMB-10Xv2-Isocortex-4
 - time taken:  53.20864101999996
WMB-10Xv2-MB
 - time taken:  3.888802752999709
WMB-10Xv2-OLF
 - time taken:  25.33346757900017
WMB-10Xv2-TH
 - time taken:  20.32103999999981
WMB-10Xv3-CB
 - time taken:  30.14816797000003
WMB-10Xv3-CTXsp
 - time taken:  16.781590239000252
WMB-10Xv3-HPF
 - time taken:  41.166393484999844
WMB-10Xv3-HY
 - time taken:  38.866329742999824
WMB-10Xv3-Isocortex-1
 - time taken:  67.22688507299972
WMB-10Xv3-Isocortex-2
 - time taken:  44.214674517000276
WMB-10Xv3-MB
 - time taken:  102.7921214779999
WMB-10Xv3-MY
 - time taken:  31.250315382999815
WMB-10Xv3-OLF
 - time taken:  12.62053

In [46]:
# change columns from index to gene symbol
gdata.columns = gene_filtered.gene_symbol
pred = pd.notna(gdata[gdata.columns[0]])
gdata = gdata[pred].copy(deep=True)
print(len(gdata))

4042976


In [47]:
os.path.join( view_directory, 'example_genes_all_cells_expression.csv')

'/alzheimer/Roberto/Allen_Institute/abc_download_root/metadata/WMB-10X/20230830/views/example_genes_all_cells_expression.csv'

In [48]:
file = os.path.join( view_directory, 'example_genes_all_cells_expression.csv')
gdata.to_csv(file)